CivicFlow AI – Prototype Notebook

Prepared by - Swaapnika Chowdary Cherukuru

A demonstration aligned with the Candidate Product Specification Memo for the AI for Impact Co-op at the Burnes Center for Social Change, Northeastern University.

This notebook showcases a mock prototype of CivicFlow AI, a generative AI-powered public records routing system that demonstrates how artificial intelligence can make government services faster, fairer, and more transparent.

It operationalizes the four-layer framework from the memo:

Trust – Accurate classification and explainable routing | Empathy – Plain-language, friendly responses | Equity – Inclusive, multilingual access | Governance – Human oversight and auditability

Because this is a simulation, all data is synthetic and no external API key is needed.
The workflow mirrors how a real civic AI product would function in partnership with a city government.

Setup and Imports

In [2]:
import pandas as pd
import random
import json

Synthetic Dataset Creation

Simulated dataset of 15 realistic citizen public records requests; covering different departments and including a Spanish example for multilingual equity.

In [3]:
requests = [
    "I would like a copy of the city council meeting minutes from March 14, 2025.",
    "There is a broken streetlight on Washington Ave near the library.",
    "I need to renew my business license for 2024.",
    "I am a journalist requesting any documents related to the budget hearings.",
    "Please provide the status of my prior public records request.",
    "I want to see the disciplinary record for a former employee.",
    "Trash collection was missed this week; when will it be picked up?",
    "I need a copy of my property tax bill for 2024.",
    "Can I get the agenda for the next council meeting?",
    "I want to appeal a parking fine; what is the process?",
    "Requesting contract documents for the new park renovation project.",
    "I want to update my business license details.",
    "How can I file a complaint against a city vendor?",
    "Please share all police incident reports from August 2025.",
    "Quiero una copia del acta de la reunión del consejo del 10 de mayo."
]

df = pd.DataFrame({
    "request_id": range(1, len(requests) + 1),
    "request_text": requests
})

df.head()

,request_id,request_text
0,1,I would like a copy of the city council meetin...
1,2,There is a broken streetlight on Washington Av...
2,3,I need to renew my business license for 2024.
3,4,I am a journalist requesting any documents rel...
4,5,Please provide the status of my prior public r...


Trust Layer – AI Classification Simulation

This cell simulates the “AI Intake Classifier & Router” described in the memo.

Each request is interpreted to determine department, intent, sensitivity, priority, and reasoning.

In [4]:
DEPARTMENTS = [
    "City Clerk",
    "Public Works",
    "Licensing and Permits",
    "Finance and Tax",
    "Legal and Public Records",
    "HR and Personnel",
    "Mayor and Council Office"
]

def mock_classify(text: str):
    t = text.lower()

    if "meeting minutes" in t or "agenda" in t or "council" in t or "acta de la reunión" in t:
        return {
            "department": "City Clerk",
            "request_intent": "request for document",
            "sensitivity_level": "low",
            "priority": "routine",
            "explanation": "The resident is asking for council records which are maintained by the City Clerk."
        }

    if "streetlight" in t or "trash" in t or "garbage" in t or "picked up" in t or "park renovation" in t or "broken" in t:
        return {
            "department": "Public Works",
            "request_intent": "complaint or service issue",
            "sensitivity_level": "low",
            "priority": "routine",
            "explanation": "The message is about a city service or public infrastructure so it goes to Public Works."
        }

    if "permit" in t or "license" in t or "food truck" in t or "business" in t:
        return {
            "department": "Licensing and Permits",
            "request_intent": "request for permit or license",
            "sensitivity_level": "low",
            "priority": "routine",
            "explanation": "The resident is asking about permits or business licensing which is handled by Licensing and Permits."
        }

    if "tax" in t or "property tax" in t or "bill" in t or "budget" in t:
        return {
            "department": "Finance and Tax",
            "request_intent": "request for billing or financial information",
            "sensitivity_level": "low",
            "priority": "routine",
            "explanation": "The request relates to financial, billing, or budgeting information."
        }

    if "journalist" in t or "reporter" in t or "press" in t or "media" in t:
        return {
            "department": "Mayor and Council Office",
            "request_intent": "media or journalist request",
            "sensitivity_level": "medium",
            "priority": "time sensitive",
            "explanation": "The requester identified as media so it is routed to the Mayor and Council Office."
        }

    if "disciplinary" in t or "police" in t or "incident reports" in t or "investigation" in t:
        return {
            "department": "Legal and Public Records",
            "request_intent": "request for document",
            "sensitivity_level": "high",
            "priority": "time sensitive",
            "explanation": "The request is for sensitive legal or personnel information which must be reviewed by Legal and Public Records."
        }

    if "status" in t or "follow up" in t or "follow-up" in t:
        return {
            "department": "City Clerk",
            "request_intent": "request for status",
            "sensitivity_level": "low",
            "priority": "routine",
            "explanation": "The citizen is following up on an earlier request so it goes to the intake team."
        }

    return {
        "department": "City Clerk",
        "request_intent": "request for document",
        "sensitivity_level": "low",
        "priority": "routine",
        "explanation": "Default route to City Clerk for general public records requests."
    }

df["model_output"] = df["request_text"].apply(mock_classify)
df[["request_id", "request_text", "model_output"]]

,request_id,request_text,model_output
0,1,I would like a copy of the city council meetin...,"{'department': 'City Clerk', 'request_intent':..."
1,2,There is a broken streetlight on Washington Av...,"{'department': 'Public Works', 'request_intent..."
2,3,I need to renew my business license for 2024.,"{'department': 'Licensing and Permits', 'reque..."
3,4,I am a journalist requesting any documents rel...,"{'department': 'Finance and Tax', 'request_int..."
4,5,Please provide the status of my prior public r...,"{'department': 'City Clerk', 'request_intent':..."
5,6,I want to see the disciplinary record for a fo...,"{'department': 'Legal and Public Records', 're..."
6,7,Trash collection was missed this week; when wi...,"{'department': 'Public Works', 'request_intent..."
7,8,I need a copy of my property tax bill for 2024.,"{'department': 'Finance and Tax', 'request_int..."
8,9,Can I get the agenda for the next council meet...,"{'department': 'City Clerk', 'request_intent':..."
9,10,I want to appeal a parking fine; what is the p...,"{'department': 'City Clerk', 'request_intent':..."


Empathy Layer – AI Reply Generation

The “AI Draft Response Assistant” from the memo.

This creates short, clear, human-friendly replies reflecting government tone.

In [5]:
def mock_reply(department: str):
    return (
        f"Hello, your request has been received and routed to the {department} team. "
        "They will review the details and follow up if any additional information is needed. "
        "Thank you for contacting the city."
    )

df["ai_reply"] = df["model_output"].apply(lambda x: mock_reply(x["department"]))
df[["request_text", "model_output", "ai_reply"]]

,request_text,model_output,ai_reply
0,I would like a copy of the city council meetin...,"{'department': 'City Clerk', 'request_intent':...","Hello, your request has been received and rout..."
1,There is a broken streetlight on Washington Av...,"{'department': 'Public Works', 'request_intent...","Hello, your request has been received and rout..."
2,I need to renew my business license for 2024.,"{'department': 'Licensing and Permits', 'reque...","Hello, your request has been received and rout..."
3,I am a journalist requesting any documents rel...,"{'department': 'Finance and Tax', 'request_int...","Hello, your request has been received and rout..."
4,Please provide the status of my prior public r...,"{'department': 'City Clerk', 'request_intent':...","Hello, your request has been received and rout..."
5,I want to see the disciplinary record for a fo...,"{'department': 'Legal and Public Records', 're...","Hello, your request has been received and rout..."
6,Trash collection was missed this week; when wi...,"{'department': 'Public Works', 'request_intent...","Hello, your request has been received and rout..."
7,I need a copy of my property tax bill for 2024.,"{'department': 'Finance and Tax', 'request_int...","Hello, your request has been received and rout..."
8,Can I get the agenda for the next council meet...,"{'department': 'City Clerk', 'request_intent':...","Hello, your request has been received and rout..."
9,I want to appeal a parking fine; what is the p...,"{'department': 'City Clerk', 'request_intent':...","Hello, your request has been received and rout..."


Governance Layer – Human-in-the-Loop Simulation

This simulates internal staff review and agreement with AI decisions, ensuring accountability and trust.

In [6]:
feedback_rows = []
for _, row in df.iterrows():
    ai_dept = row["model_output"]["department"]
    human_dept = ai_dept  # simulate approval
    feedback_rows.append({
        "request_id": row["request_id"],
        "ai_department": ai_dept,
        "human_department": human_dept,
        "agreement": ai_dept == human_dept
    })

feedback_df = pd.DataFrame(feedback_rows)
feedback_df

,request_id,ai_department,human_department,agreement
0,1,City Clerk,City Clerk,True
1,2,Public Works,Public Works,True
2,3,Licensing and Permits,Licensing and Permits,True
3,4,Finance and Tax,Finance and Tax,True
4,5,City Clerk,City Clerk,True
5,6,Legal and Public Records,Legal and Public Records,True
6,7,Public Works,Public Works,True
7,8,Finance and Tax,Finance and Tax,True
8,9,City Clerk,City Clerk,True
9,10,City Clerk,City Clerk,True


Metrics Dashboard

Simulated performance metrics and analytics aligned with the memo’s “Success Metrics” section.

In [7]:
alignment_rate = feedback_df["agreement"].mean()
dept_counts = feedback_df["ai_department"].value_counts()

print("Human–AI alignment rate:", f"{alignment_rate:.0%}")
print("\nPredicted departments distribution:\n")
print(dept_counts)

Human–AI alignment rate: 100%

Predicted departments distribution:

ai_department
City Clerk                  6
Public Works                3
Licensing and Permits       2
Finance and Tax             2
Legal and Public Records    2
Name: count, dtype: int64


Data Export for Transparency and Audit

Just as in the memo, CivicFlow AI maintains traceability through structured exports.

In [8]:
df.to_json("civicflow_predictions_demo.json", orient="records", indent=2)
feedback_df.to_csv("civicflow_feedback_demo.csv", index=False)

print("Saved civicflow_predictions_demo.json and civicflow_feedback_demo.csv")

Saved civicflow_predictions_demo.json and civicflow_feedback_demo.csv


Final Clean Display Table

Final view
It shows all fields clearly, matching the memo’s structure: department, intent, sensitivity, priority, and explanation.

In [9]:
display_df = df[["request_id", "request_text"]].copy()
display_df["department"] = df["model_output"].apply(lambda x: x["department"])
display_df["intent"] = df["model_output"].apply(lambda x: x["request_intent"])
display_df["sensitivity"] = df["model_output"].apply(lambda x: x["sensitivity_level"])
display_df["priority"] = df["model_output"].apply(lambda x: x["priority"])
display_df["explanation"] = df["model_output"].apply(lambda x: x["explanation"])
display_df

,request_id,request_text,department,intent,sensitivity,priority,explanation
0,1,I would like a copy of the city council meetin...,City Clerk,request for document,low,routine,The resident is asking for council records whi...
1,2,There is a broken streetlight on Washington Av...,Public Works,complaint or service issue,low,routine,The message is about a city service or public ...
2,3,I need to renew my business license for 2024.,Licensing and Permits,request for permit or license,low,routine,The resident is asking about permits or busine...
3,4,I am a journalist requesting any documents rel...,Finance and Tax,request for billing or financial information,low,routine,"The request relates to financial, billing, or ..."
4,5,Please provide the status of my prior public r...,City Clerk,request for status,low,routine,The citizen is following up on an earlier requ...
5,6,I want to see the disciplinary record for a fo...,Legal and Public Records,request for document,high,time sensitive,The request is for sensitive legal or personne...
6,7,Trash collection was missed this week; when wi...,Public Works,complaint or service issue,low,routine,The message is about a city service or public ...
7,8,I need a copy of my property tax bill for 2024.,Finance and Tax,request for billing or financial information,low,routine,"The request relates to financial, billing, or ..."
8,9,Can I get the agenda for the next council meet...,City Clerk,request for document,low,routine,The resident is asking for council records whi...
9,10,I want to appeal a parking fine; what is the p...,City Clerk,request for document,low,routine,Default route to City Clerk for general public...


### Prototype Summary

This notebook demonstrates how CivicFlow AI translates the Candidate Product Specification Memo into an actionable prototype.

**Four-Layer Framework**
- **Trust Layer:** Classifies and explains routing decisions.
- **Empathy Layer:** Generates polite, clear, plain-language responses.
- **Equity Layer:** Handles multilingual inputs (e.g., Spanish request).
- **Governance Layer:** Records human reviews for transparency and improvement.

**How It Reflects the Memo**
- **Phase 1 (AI Intake Classifier)** and **Phase 2 (AI Response Assistant)** implemented.  
- Uses mock data to simulate real-world workflows.  
- Demonstrates the technical feasibility, ethical design, and measurable benefits outlined in the memo.

---

### Roadmap & Next Steps

**Phase 0 – Data Readiness:** Prepare and anonymize two years of real request data for model fine-tuning.  
**Phase 1 – Classifier Pilot:** Deploy the CivicFlow AI intake module (demonstrated in this notebook).  
**Phase 2 – Response Assistant:** Expand to generate context-aware communication templates.  
**Phase 3 – Public Self-Service Portal:** Build citizen-facing search functionality to reduce request volume.

---

### Key Performance Metrics (Simulated Targets)
| Metric | Target | Purpose |
|--------|--------|----------|
| Processing efficiency | +30% | Faster routing and response times |
| Classification accuracy | 95% | Reliable department routing |
| Human-AI alignment | 100% (demo) | Builds staff trust |
| Compliance incidents | 0 | Maintain transparency and privacy |
| Accessibility improvement | +25% | Support for multilingual users |

---

### Research Foundations

This project draws inspiration from real-world civic AI initiatives:
- **San Francisco’s 311 AI Request Triage System**  
- **New Jersey’s FAIR (Fast AI-Assisted Investigation & Review)**  
- **Massachusetts DOT’s HEKA AI for Infrastructure Monitoring**

These projects informed the system’s focus on **responsibility, explainability, and inclusion** in AI for public service.

---

### Equity and Impact Statement

By transforming public record management into an inclusive, transparent, and data-driven process,  
CivicFlow AI ensures that **every resident, regardless of background or language**, can engage with their government on equal footing.  
This model demonstrates how **generative AI can enhance - not replace - human empathy, efficiency, and trust** in civic systems.